In [13]:
from functools import reduce
import re
import pandas as pd
import numpy as np
from scipy.spatial.distance import mahalanobis
from scipy.optimize import linear_sum_assignment
import warnings

warnings.filterwarnings('ignore')

ЗДЕСЬ И СОБЫТИЯ ЯНДЕКСА И РОССТАТ

In [14]:
events1 = pd.read_csv('события.csv')
events2 = pd.read_csv('росстат_общий_new1.csv')


df = events2.merge(
    events1[['Event Text', 'region_name', 'municipality', 'year']],
    on=['municipality', 'region_name', 'year'],
    how='left'
)


df.to_csv('общие_данные_2010t5+.csv', index=False)
df

,year,region_name,municipality,oktmo,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),Население,Прирост,...,Бюджет,Рождаемость,Смертность,Родившихся,Инвестиции,СреднегодН,Квартиры,Зарплата,Земля,Event Text
0,2006,Тверская,NaN,28602100,0.0,0.0,0.0,0.0,0.0,0.0,...,-433.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,2006,Тверская,NaN,28602400,0.0,0.0,0.0,0.0,0.0,0.0,...,284.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,2006,Тверская,NaN,28604100,0.0,0.0,0.0,0.0,0.0,0.0,...,-375.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,2006,Тверская,NaN,28604400,0.0,0.0,0.0,0.0,0.0,0.0,...,345.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,2006,Тверская,NaN,28606100,0.0,0.0,0.0,0.0,0.0,0.0,...,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
797868,2024,Санкт-Петербург,№,40322000,0.0,0.0,0.0,0.0,191571.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
797869,2024,Санкт-Петербург,№,40331000,0.0,0.0,0.0,0.0,76567.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
797870,2024,Санкт-Петербург,№,40383000,0.0,0.0,0.0,0.0,67469.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
797871,2024,Санкт-Петербург,№,40903000,0.0,0.0,0.0,0.0,67637.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


ОБЪЕДИНЕНИЕ ПРОТЕСТОВ С ДАННЫМИ РОССТАТА 

1. ВООБЩЕ 
2. ТОЛЬКО ЗА 2012

In [15]:
data = pd.read_csv('общие_данные_2010t5+.csv')

protest_events = ["Demonstrate or rally", "Criticize or denounce", "Defy norms, law"]


protests = data[data['Event Text'].isin(protest_events)].copy()
protests_0910 = protests[protests['year'].isin([2009, 2010, 2011, 2012])].copy()

# Для каждого oktmo записываем минимальный год протеста в 2009–2010
protests_0910['fprotets_year'] = protests_0910.groupby('oktmo')['year'].transform('min')

# Оставляем по одной строке на oktmo (колонка fprotets_year уже заполнена правильно)
protests1 = protests_0910.drop_duplicates(subset=['oktmo']).reset_index(drop=True)



protests.to_csv('протесты_2010t5+.csv', index=False)
protests1.to_csv('протесты2009_2013_2010t5+.csv', index=False)
protests1

,year,region_name,municipality,oktmo,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),Население,Прирост,...,Рождаемость,Смертность,Родившихся,Инвестиции,СреднегодН,Квартиры,Зарплата,Земля,Event Text,fprotets_year
0,2009,Хакасия,Абакан,95701000,2.79,0.00,400.0,860.0,163362.0,0.0,...,0.0,0.0,0.0,0.0,163490.0,416.00,20553.0,11238.0,Criticize or denounce,2009
1,2009,Башкортостан,Альшеевский,80602000,0.80,0.00,200.0,3100.0,44384.0,0.0,...,0.0,0.0,0.0,0.0,44300.0,416.00,10346.3,241142.0,Criticize or denounce,2009
2,2009,Краснодарский,Анапа,3703000,0.90,2.22,300.0,21300.0,139761.0,0.0,...,0.0,0.0,0.0,0.0,141105.0,366.00,14102.0,98186.0,Criticize or denounce,2009
3,2009,Тверская,Андреапольский,28602000,15.60,0.00,0.0,0.0,13970.0,0.0,...,0.0,0.0,0.0,0.0,13792.0,618.06,10502.7,305086.0,Criticize or denounce,2009
4,2009,Архангельская,Архангельск,11701000,485.50,0.00,2000.0,1300.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,416.00,22976.9,29445.0,Criticize or denounce,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,2012,Чувашская,Чебоксарский,97644000,11.20,41.61,0.0,5200.0,63314.0,0.0,...,0.0,0.0,0.0,10994.1,63321.0,0.00,15145.8,117884.0,Demonstrate or rally,2012
167,2012,Московская,Шаховская,46658151,14.60,0.00,68.0,270.0,11545.0,-8.0,...,10.0,18.0,120.0,0.0,11532.0,0.00,0.0,3895.0,Demonstrate or rally,2012
168,2012,Ивановская,Шуйский,24633000,11.00,43.58,90.0,650.0,21791.0,-6.0,...,11.0,16.0,234.0,0.0,21802.0,0.00,11207.7,106454.0,Demonstrate or rally,2012
169,2012,Калмыкия,Элиста,85701000,38.30,51.14,44.0,4377.0,108667.0,6.3,...,16.0,9.7,1744.0,0.0,108753.0,0.00,18923.8,39657.0,Criticize or denounce,2012


ЕДИНИЦЫ БЕЗ ПРОТЕСТОВ СРЕДИ ВСЕХ СОБЫТИЙ ЗА ВСЕ ВРЕМЯ

In [16]:
lol = pd.read_csv("протесты_2010t5+.csv") 
sobytia = pd.read_csv("общие_данные_2010t5+.csv")


noprotestALL = sobytia[~sobytia['municipality'].isin(lol['municipality'])].reset_index(drop=True)
noprotestALL.to_csv('единицы_без_протестов_2010t5+.csv', index=False)
noprotestALL

,year,region_name,municipality,oktmo,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),Население,Прирост,...,Бюджет,Рождаемость,Смертность,Родившихся,Инвестиции,СреднегодН,Квартиры,Зарплата,Земля,Event Text
0,2006,Тверская,NaN,28602100,0.0,0.0,0.0,0.0,0.0,0.0,...,-433.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,2006,Тверская,NaN,28602400,0.0,0.0,0.0,0.0,0.0,0.0,...,284.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,2006,Тверская,NaN,28604100,0.0,0.0,0.0,0.0,0.0,0.0,...,-375.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,2006,Тверская,NaN,28604400,0.0,0.0,0.0,0.0,0.0,0.0,...,345.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,2006,Тверская,NaN,28606100,0.0,0.0,0.0,0.0,0.0,0.0,...,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623222,2024,Санкт-Петербург,№,40322000,0.0,0.0,0.0,0.0,191571.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
623223,2024,Санкт-Петербург,№,40331000,0.0,0.0,0.0,0.0,76567.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
623224,2024,Санкт-Петербург,№,40383000,0.0,0.0,0.0,0.0,67469.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
623225,2024,Санкт-Петербург,№,40903000,0.0,0.0,0.0,0.0,67637.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


ПОИСК КОНТРОЛЬНОЙ ГРУППЫ (НЕПРОТЕСТНЫЕ ЕДИНИЦЫ ЗА 2011 ИЗ ТЕХ ЧТО ПРОТЕСТНЫЕ В 2012)

In [17]:
lol = pd.read_csv("протесты2009_2013_2010t5+.csv") 
sobytia = pd.read_csv("общие_данные_2010t5+.csv")

df_2011 = sobytia[sobytia['year'] == 2008]


filtered_df = df_2011[df_2011['oktmo'].isin(lol['oktmo'])]

pattern = "Demonstrate or rally|Criticize or denounce|Defy norms, law"
mask = filtered_df['Event Text'].str.contains(pattern, case=False, na=False)
municipalities_with_tag = filtered_df[mask]['oktmo'].unique()


final_df = filtered_df[~filtered_df['oktmo'].isin(municipalities_with_tag)].reset_index(drop=True)

final_df = final_df.drop_duplicates(subset='oktmo').reset_index(drop=True)
final_df.to_csv('без_протестов2008_2010t5+.csv', index=False)
final_df



,year,region_name,municipality,oktmo,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),Население,Прирост,...,Бюджет,Рождаемость,Смертность,Родившихся,Инвестиции,СреднегодН,Квартиры,Зарплата,Земля,Event Text
0,2008,Хакасия,Абакан,95701000,21.1,0.00,381.0,6300.0,0.0,0.0,...,-13052.0,0.0,0.0,0.0,0.0,0.0,411.00,18275.2,11238.0,"Arrest, detain, or charge with legal action"
1,2008,Ставропольский,Александровский,7602000,0.0,0.00,0.0,0.0,0.0,-3.6,...,6993.0,12.6,16.2,632.0,0.0,0.0,359.48,7463.3,0.0,NaN
2,2008,Ставропольский,Александровский,7602402,0.0,1.46,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,7866.5,37450.0,NaN
3,2008,Удмуртская,Алнашский,94602000,0.0,0.00,0.0,2535.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.34,7230.7,89600.0,Make statement
4,2008,Краснодарский,Анапа,3703000,1.0,11.30,1600.0,34000.0,0.0,0.0,...,-41505.0,0.0,0.0,0.0,0.0,0.0,360.00,12470.0,98186.0,"Arrest, detain, or charge with legal action"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,2008,Калмыкия,Элиста,85701000,51.8,0.00,0.0,2700.0,106598.0,0.0,...,25071.6,0.0,0.0,0.0,0.0,0.0,0.31,11386.3,39657.0,Express intent to meet or negotiate
132,2008,Саха,Якутск,98701000,328.5,2.16,4279.0,3192.0,0.0,0.0,...,-123850.0,0.0,0.0,0.0,0.0,0.0,302.00,26440.7,355334.0,"Arrest, detain, or charge with legal action"
133,2008,Башкортостан,Янаульский,80659000,4.5,0.00,0.0,1500.0,0.0,0.0,...,-1493.0,0.0,0.0,0.0,0.0,0.0,413.00,14489.6,209381.0,NaN
134,2008,Ярославская,Ярославль,78701000,325.2,0.00,16900.0,27900.0,0.0,0.0,...,-1445796.0,0.0,0.0,0.0,0.0,0.0,412.98,0.0,20580.0,Use unconventional violence


Создание регрессии

In [18]:
def check_years_completeness(oktmo, df_events_full, required_years):
    municipality_data = df_events_full[df_events_full['oktmo'] == oktmo]
    existing_years = set(municipality_data['year'].unique())
    return existing_years.issuperset(required_years)

df_protests = pd.read_csv("протесты2009_2013_2010t5+.csv")
df_protests = df_protests.drop_duplicates(subset=['region_name', 'municipality'])
df_events = pd.read_csv("единицы_без_протестов_2010t5+.csv")
df_events_full = pd.read_csv("общие_данные_2010t5+.csv")

required_years = {2008.0, 2013.0, 2014.0, 2015.0}


df_events = df_events[df_events['year'].isin([2009, 2010, 2011, 2012])].reset_index(drop=True)

features = [
            'ПотреблениеЭ', 
            'ПотреблениеВ', 
            'Бюджет',
            'БезДорог', 
            'Земля',
            # 'Квартиры',

           
            # 'Население', 
            # 'Прибыль',
            # 'Зарплата',
            # 'Расходы',

            # 'СреднегодН',
            # 'ПотреблениеТ', 
            # 'ЭконАктив'
            # 'Прирост', 
            # 'Рождаемость',
            # 'Смертность',
            # 'Родившихся',
            # 'Инвестиции',
]

df_protests = df_protests[
    ['region_name', 'municipality', 'oktmo', 'fprotets_year', "Жилье (1000 м2)", "Дороги (%)", "Канализация (1 м)",
    "Водопровод (1 метр)"] + features
].reset_index(drop=True)
df_events = df_events[
    ['region_name', 'municipality', 'oktmo', "Жилье (1000 м2)", "Дороги (%)", "Канализация (1 м)",
    "Водопровод (1 метр)"] + features
].reset_index(drop=True)

exclude_dict = {
        'Абакан': ['Усть-Абаканский', 'Алтайский', 'Минусинский', 'Минусинск', 'Черногорск'],
    'Архангельск': ['Северодвинск', 'Приморский'],
    'Брянск': ['Брянский', 'Сельцо', 'Фокино', 'Титовское'],
    'Вологда': ['Вологодский'],
    'Тула': ['Щекинский', 'Дубенский', 'Алексин', 'Ясногорский', 'Веневский', 'Киреевский', 'Ленинский'],
    'Ярославль': ['Ярославский'],
    'Казань': ['Лаишевский', 'Пестречинский', 'Высокогорский', 'Зеленодольский', 'Верхнеуслонский', 'Новоильмовское'],
    'Омск': ['Омский'],
    'Оренбург': ['Оренбургский', 'Сакмарский'],
    'Тамбов': ['Тамбовский', 'Котовск'],
    'Иркутск': ['Иркутский', 'Ангарское', 'Ангарск', 'Шелеховский'],
    'Калуга': ['Перемышльский', 'Ферзиковский', 'Малоярославецкий', 'Дзержинский', 'Бабынинский'],
    'Анапа': ['Темрюкский', 'Крымский', 'Новороссийск'],
    'Балашиха': ['Королев', 'Люберцы', 'Богородский', 'Щелково', 'Раменский'],
    'Бокситогорский': ['Тихвинский', 'Любытинский', 'Хвойнинский', 'Чагодощенский', 'Бабаевский'],
    'Варгашинский': ['Кетовский', 'Курган', 'Половинский', 'Лебяжьевский', 'Мокроусовский', 'Упоровский', 'Белозерский'],
    'Горно-Алтайск': ['Майминский'],
    'Добрянский': ['Краснокамский', 'Ильинский', 'Юсьвинский', 'Березники', 'Александровский', 'Губаха', 'Чусовской', 'Пермский'],
    'Кола': ['Североморск', 'Александровск', 'Кольский', 'Кильдинстрой'],
    'Комсомольск-на-Амуре': ['Комсомольский'],
    'Краснодар': ['Динский', 'Красноармейский', 'Тахтамукайский'],
    'Красноярский': ['Володарский', 'Приволжский', 'Наримановский', 'Харабалинский'],
    'Курганинский': ['Лабинский', 'Новокубанский', 'Гулькевичский', 'Тбилисский', 'Шовгеновский', 'Кошехабльский'],
    'Новороссийск': ['Новороссийский'],
    'Орёл': ['Орловский'],
    'Почепский': ['Трубчевский', 'Погарский', 'Унечский', 'Мглинский', 'Клетнянский', 'Жирятинский', 'Выгоничский', 'Титовское'],
    'Ростов': ['Ростовский'],
    'Саратовский': ['Красноармейский', 'Саратов', 'Лысогорский', 'Татищевский'],
    'Тольятти': ['Жигулевск', 'Ставропольский'],
    'Учалинский': ['Белорецкий', 'Катав-Ивановский', 'Верхнеуральский', 'Уйский', 'Чебаркульский', 'Миасский', 'Златоустовский'],
    'Целинский': ['Егорлыкский', 'Зерноградский', 'Сальский', 'Песчанокопский', 'Белоглинский'],
    'Абазинский': ['Хабезский', 'Усть-Джегутинский', 'Черкесский', 'Прикубанский', 'Адыге-Хабльский'],
    'Алнашский': ['Граховский', 'Менделеевский', 'Агрызский', 'Можгинский'],
    'Апатиты': ['Полярные Зори', 'Кандалакшский', 'Терский', 'Кировск', 'Оленегорск', 'Мончегорск'],
    'Архаринский': ['Облученский', 'Верхнебуреинский', 'Бурейский'],
    'Асекеевский': ['Грачёвский', 'Матвеевский', 'Абдулинский', 'Бугурусланский', 'Бузулукский'],
    'Аургазинский': ['Кармаскалинский', 'Давлекановский', 'Альшеевский', 'Стерлитамакский', 'Гафурийский'],
    'Бабаюрт': ['Шелковской', 'Хасавюртовский', 'Кизилюртовский', 'Кумторкалинский', 'Кизлярский'],
    'Баксан': ['Баксанский', 'Чегемский', 'Прохладненский'],
    'Баксанский': ['Баксан', 'Чегемский', 'Прохладненский', 'Зольский', 'Эльбрусский', 'Кировский'],
    'Бердск': ['Новосибирский', 'Искитим', 'Искитимский', 'Кольцово'],
    'Благовещенск': ['Благовещенский', 'Кушнаревский', 'Имангуловский'],
    'Буйнакск': ['Буйнакский'],
    'Воловский': ['Тепло-Огаревский', 'Каменский', 'Ефремов', 'Куркинский', 'Богородицкий'],
    'Волосовский': ['Кингисеппский', 'Сланцевский', 'Лужский', 'Гатчинский', 'Ломоносовский'],
    'Воронеж': ['Хохольский', 'Семилукский', 'Рамонский', 'Новоусманский', 'Каширский'],
    'Гатчинский': ['Лужский', 'Волосовский', 'Тосненский', 'Пушкин', 'Ломоносовский'],
    'Геленджик': ['Туапсинский', 'Северский', 'Абинский', 'Крымский', 'Новороссийск'],
    'Грязовецкий': ['Вологодский', 'Междуреченский', 'Солигаличский', 'Буйский', 'Любимский', 'Первомайский', 'Пошехонский'],
    'Гулькевичский': ['Кавказский', 'Новоалександровский', 'Новокубанский', 'Курганинский', 'Тбилисский'],
    'Гурьевский': ['Зеленоградский', 'Калининград', 'Полесский', 'Гвардейский', 'Багратионовский', 'Ладушкинский'],
    'Даниловский': ['Котовский', 'Руднянский', 'Еланский', 'Михайловка', 'Фроловский', 'Ольховский'],
    'Дербент': ['Дербентский', 'Докузпаринский', 'Курахский', 'Дагестанские Огни'],
    'Дмитриевский': ['Хомутовский', 'Севский', 'Комаричский', 'Дмитровский', 'Железногорский', 'Конышевский'],
    'Домбаровский': ['Ясненский', 'Комаровский', 'Новоорский', 'Орск'],
    'Домодедово': ['Подольск', 'Ленинский', 'Раменский', 'Чехов', 'Ступино', 'Щербинка'],
    'Дубна': ['Кимрский', 'Кимры', 'Талдомский', 'Дмитровский', 'Конаковский'],
    'Жуковский': ['Раменский', 'Лыткарино', 'Люберцы', 'Бронницы'],
    'Завьяловский': ['Ижевск', 'Увинский', 'Малопургинский', 'Сарапульский', 'Чайковский', 'Воткинский', 'Якшур-Бодьинский'],
    'Залегощенский': ['Орёл', 'Орловский', 'Свердловский', 'Покровский', 'Верховский', 'Новосильский', 'Мценский'],
    'Заполярный': ['Мезенский', 'Усть-Цилемский', 'Усинск', 'Инта', 'Воркута'],
    'Ивановский': ['Иваново', 'Фурмановский', 'Комсомольский', 'Тейковский', 'Лежневский', 'Шуйский', 'Родниковский'],
    'Избербаш': ['Каякентский', 'Сергокалинский', 'Дахадаевский', 'Карабудахкентский'],
    'Кашарский': ['Миллеровский', 'Чертковский', 'Верхнедонской', 'Боковский', 'Советский', 'Милютинский', 'Тарасовский'],
    'Кизильский': ['Карталинский', 'Брединский', 'Кваркенский', 'Баймакский', 'Сибай', 'Абзелиловский', 'Агаповский'],
    'Кизляр': ['Шелковской', 'Кизлярский', 'Тарумовский', 'Бабаюртовский', 'Цунтинский', 'Тляратинский', 'Гумбетовский'],
    'Кингисеппский': ['Сланцевский', 'Волосовский', 'Ломоносовский', 'Сосновоборский'],
    'Кинель': ['Волжский', 'Кинельский'],
    'Кинельский': ['Кинель', 'Волжский', 'Красноярский', 'Кинель-Черкасский', 'Богатовский', 'Нефтегорский', 'Приволжье'],
    'Кировский': ['Куйбышевский', 'Дятьковский', 'Людиновский', 'Думиничский', 'Сухиничский', 'Барятинский', 'Спас-Деменский'],
    'Кисловодск': ['Предгорный', 'Малокарачаевский', 'Зольский'],
    'Комаричский': ['Суземский', 'Севский', 'Дмитриевский', 'Дмитровский', 'Брасовский'],
    'Крестецкий': ['Новгородский', 'Парфинский', 'Демянский', 'Валдайский', 'Окуловский', 'Маловишерский'],
    'Курумканский': ['Северо-Байкальский', 'Баунтовский', 'Баргузинский'],
    'Ленинский': ['Домодедово', 'Подольск', 'Раменский', 'Лыткарино', 'Дзержинский', 'Подольск'],
    'Лермонтов': ['Предгорный', 'Железноводск', 'Пятигорск', 'Ессентуки', 'Минераловодский'],
    'Ловозерский': ['Терский', 'Кировск', 'Оленегорск', 'Кольский'],
    'Лыткарино': ['Люберцы', 'Раменский', 'Ленинский', 'Дзержинский'],
    'Магадан': ['Ольский', 'Хасынский'],
    'Макаровский': ['Долинский', 'Томаринский', 'Углегорский', 'Поронайский', 'Вахрушев'],
    'Мегион': ['Нижневартовский', 'Нижневартовск', 'Лангепас'],
    'Мезенский': ['Заполярный', 'Усть-Цилемский', 'Лешуконский', 'Пинежский', 'Приморский', 'Урдомское'],
    'Михайловский': ['Новомосковск', 'Кимовский', 'Скопинский', 'Пронский', 'Захаровский', 'Серебряные Пруды'],
    'Надымский': ['Пуровский', 'Сургутский', 'Белоярский', 'Приуральский', 'Ямальский', 'Тазовский', 'Новый Уренгой'],
    'Невинномысск': ['Кочубеевский'],
    'Некоузский': ['Мышкинский', 'Сонковский', 'Краснохолмский', 'Брейтовский', 'Рыбинский'],
    'Новгородский': ['Старорусский', 'Парфинский', 'Крестецкий', 'Маловишерский', 'Чудовский', 'Тосненский', 'Лужский', 'Батецкий', 'Шимский'],
    'Нягань': ['Октябрьский'],
    'Одинцовский': ['Власиха', 'Внуковское', 'Кокошкинское', 'Можайский', 'Красногорск', 'Марушкинское', 'Наро-Фоминский', 'Рузский', 'Истра'],
    'Палласовский': ['Быковский', 'Николаевский', 'Старополтавский', 'Знаменск', 'Жанибекский'],
    'Пинежский': ['Лешуконский', 'Мезенский', 'Приморский', 'Холмогорский', 'Виноградовский', 'Верхнетоемский', 'Удорский'],
    'Прибайкальский': ['Кабанский', 'Иволгинский', 'Улан-Удэ', 'Заиграевский', 'Хоринский', 'Баргузинский', 'Усть-Джилиндинское'],
    'Пушкинский': ['Мытищи', 'Дмитровский', 'Сергиево-Посадский', 'Красноармейск', 'Щёлково', 'Восточный', 'Ивантеевка'],
    'Салехард': ['Приуральский', 'Шурышкарский'],
    'Северо-Курильский': ['Курильский', 'Елизовский', 'Усть-Большерецкий'],
    'Северо-Эвенский': ['Пенжинский', 'Карагинский', 'Билибинский', 'Среднеканский', 'Омсукчанский'],
    'Северодвинск': ['Приморский', 'Архангельск'],
    'Северский': ['Туапсинский', 'Геленджик', 'Абинский', 'Горячий Ключ', 'Тахтамукайский', 'Красноармейский'],
    'Серафимовичский': ['Советский', 'Боковский', 'Шолоховский', 'Кумылженский', 'Михайловка', 'Фроловский', 'Клетский'],
    'Сергиевский': ['Красноярский', 'Елховский', 'Кошкинский', 'Челно-Вершинский', 'Шенталинский', 'Похвистневский', 'Кинель-Черкасский'],
    'Слюдянский': ['Закаменский', 'Тункинский', 'Кабанский', 'Иркутский', 'Шелеховский', 'Усольский', 'Баяндаевский'],
    'Солигаличский': ['Буйский', 'Галичский', 'Чухломский', 'Бабушкинский', 'Тотемский', 'Междуреченский', 'Грязовецкий'],
    'Сыктывдинский': ['Сыктывкар', 'Сысольский', 'Ленский', 'Усть-Вымский', 'Княжпогостский', 'Корткеросский', 'Койгородский'],
    'Таштыпский': ['Абаза', 'Таштагольский', 'Турочакский', 'Улаганский', 'Бай-Тайгинский', 'Барун-Хемчикский', 'Сут-Хольский', 'Шушенский', 'Бейский', 'Аскизский'],
    'Тимашевский': ['Кореновский', 'Динский', 'Калининский', 'Приморско-Ахтарский', 'Брюховецкий'],
    'Тобольск': ['Тобольский'],
    'Торжокский': ['Кувшиновский', 'Старицкий', 'Калининский', 'Лихославльский', 'Спировский', 'Вышневолоцкий', 'Шепелевское'],
    'Торопецкий': ['Западнодвинский', 'Андреапольский', 'Холмский', 'Локнянский', 'Великолукский', 'Куньинский'],
    'Тотемский': ['Бабушкинский', 'Чухломский', 'Солигаличский', 'Междуреченский', 'Сокольский', 'Сямженский', 'Верховажский', 'Тарногский', 'Нюксенский'],
    'Тюмень': ['Тюменский', 'Нижнетавдинский'],
    'Хасавюрт': ['Хасавюртовский', 'Новолакский', 'Ахвахский'],
    'Хвойнинский': ['Боровичский', 'Мошенской', 'Пестовский', 'Чагодощенский', 'Бокситогорский', 'Любытинский'],
    'Холмогорский': ['Приморский', 'Пинежский', 'Виноградовский', 'Мирный', 'Плесецкий'],
    'Чердынский': ['Троицко-Печорский', 'Красновишерский', 'Соликамский', 'Косинский', 'Гайнский', 'Усть-Куломский'],
    'Череповец': ['Череповецкий'],
    'Чернский': ['Мценский', 'Корсаковский', 'Каменский', 'Тепло-Огаревский', 'Плавский', 'Арсеньевский', 'Славный'],
    'Шуйский': ['Шуя', 'Савинский', 'Южский', 'Палехский', 'Родниковский', 'Ивановский', 'Лежневский'],
    'Янаульский': ['Куединский', 'Татышлинский', 'Бураевский', 'Калтасинский', 'Краснокамский', 'Камбарский', 'Нефтекамск', 'Чайковский'],
    'Ярославский': ['Ярославль', 'Тутаевский', 'Большесельский', 'Борисоглебский', 'Гаврилов-Ямский', 'Некрасовский', 'Даниловский'],
    'Архангельский':       ['Гафурийский', 'Белорецкий', 'Ашинский', 'Иглинский', 'Кармаскалинский'],
    'Березники':           ['Соликамский', 'Александровский', 'Добрянский', 'Юсьвинский', 'Кудымкарский', 'Косинский'],
    'Воловский':           ['Богородицкий', 'Куркинский', 'Ефремов', 'Каменский', 'Тепло-Огаревский'],
    'Комсомольский':       ['Нефтегорский', 'Богатовский', 'Кинель-Черкасский', 'Красноярский', 'Кинель', 'Волжский'],
    'Петропавловск‑Камчатский': ['Вилючинский', 'Елизовский'],
    'Усть-Джегутинский':   ['Усть-Джегутинское'],
    'Пятигорск':           ['Предгорный', 'Ессентуки', 'Лермонтов', 'Железноводск'],
    'Саранск':             ['Рузаевский', 'Лямбирский', 'Кочкуровский'],
    'Шаховская':           ['Можайский', 'Гагаринский', 'Зубцовский', 'Лотошинский', 'Волоколамский'],
    'Чебоксарский':        ['Чебоксары', 'Моргаушский', 'Красноармейский', 'Цивильский', 'Мариинско-Посадский', 'Звениговский', 'Килемарский'],
    'Чегемский':           ['Нальчик', 'Черекский', 'Эльбрусский', 'Баксанский', 'Баксан', 'Урванский'],
    'Элиста':              ['Ики-Бурульский', 'Приютненский', 'Целинный'],
    'Якутск':              ['Хангаласский', 'Мегино-Кангаласский', 'Горный', 'Намский', 'Усть-Алданский'],
    "Александровский": [
        "Петровский", "Благодарненский", "Новоселицкий",
        "Георгиевский", "Минераловодский", "Андроповский", "Грачёвский"
    ]
}

excluded_pairs = set()
for protest, events in exclude_dict.items():
    for event in events:
        excluded_pairs.add((protest, event))

# Проверка на вырожденные признаки
combined_all = pd.concat([df_protests[features], df_events[features]], axis=0)
variances = combined_all.var()
threshold = 1e-8
degenerate = variances[variances <= threshold].index.tolist()
if degenerate:
    print(f"Удаляем вырожденные признаки: {degenerate}")
    features = [f for f in features if f not in degenerate]
else:
    print("Вырожденных признаков нет.")

combined_all = pd.concat([df_protests[features], df_events[features]], axis=0)
cov_matrix = np.cov(combined_all.T)
cov_inv = np.linalg.inv(cov_matrix)

# Matching внутри каждого региона с проверкой ограничений
matches_list = []

for region in df_protests['region_name'].unique():
    sub_protests = df_protests[df_protests['region_name'] == region].reset_index(drop=True)
    sub_events = df_events[df_events['region_name'] == region].reset_index(drop=True)
    
    # Фильтрация событий: полнота данных + проверка на исключения
    valid_events = []
    for _, event_row in sub_events.iterrows():
        event_name = event_row['municipality']
        oktmo = event_row['oktmo']
        
        # Проверка полноты данных
        has_full_data = check_years_completeness(oktmo, df_events_full, required_years)
        
        # Проверка, что нет ни одного протеста, для которого это событие запрещено
        is_allowed = all(
            (protest_name, event_name) not in excluded_pairs
            for protest_name in sub_protests['municipality']
        )
        
        if has_full_data and is_allowed:
            valid_events.append(event_row)
    
    if not valid_events:
        print(f"В регионе {region} нет подходящих муниципалитетов событий")
        continue
    
    sub_events_valid = pd.DataFrame(valid_events).reset_index(drop=True)
    
    n_p = sub_protests.shape[0]
    n_e = sub_events_valid.shape[0]
    
    if n_e == 0:
        continue
    
    dist_mat = np.zeros((n_p, n_e))
    
    # Заполнение матрицы расстояний
    for i in range(n_p):
        x = sub_protests.loc[i, features].values
        for j in range(n_e):
            y = sub_events_valid.loc[j, features].values
            dist_mat[i, j] = mahalanobis(x, y, cov_inv)
    
    # Венгерский алгоритм
    row_ind, col_ind = linear_sum_assignment(dist_mat)
    
    # Сбор результатов
    matched_p = sub_protests.iloc[row_ind].reset_index(drop=True)
    matched_e = sub_events_valid.iloc[col_ind].reset_index(drop=True)
    
    matched = matched_p.join(
        matched_e,
        lsuffix="_protest",
        rsuffix="_event"
    )
    matched["mahalanobis_distance"] = dist_mat[row_ind, col_ind]
    matches_list.append(matched)




df_matches = pd.concat(matches_list, axis=0).reset_index(drop=True)
df_matches.to_csv('прототип_данных_2010t5+.csv', index=False)
df_matches

Вырожденных признаков нет.
В регионе Ингушетия нет подходящих муниципалитетов событий
В регионе Чеченская нет подходящих муниципалитетов событий


,region_name_protest,municipality_protest,oktmo_protest,fprotets_year,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,ПотреблениеЭ_protest,ПотреблениеВ_protest,...,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,ПотреблениеЭ_event,ПотреблениеВ_event,Бюджет_event,БезДорог_event,Земля_event,mahalanobis_distance
0,Хакасия,Абакан,95701000,2009,2.79,0.00,400.0,860.0,0.0,0.00,...,5.60,0.0,0.0,139.0,0.0,0.00,16612.0,0.00,8861.0,0.182434
1,Хакасия,Таштыпский,95625000,2010,10830.00,0.00,0.0,10.0,600.0,26.10,...,34717.00,0.0,0.0,1600.0,624.0,11.03,1174.0,0.87,661055.0,1.612928
2,Башкортостан,Альшеевский,80602000,2009,0.80,0.00,200.0,3100.0,723.0,0.00,...,5.20,0.0,0.0,1000.0,699.0,0.00,29082.0,0.00,245588.0,0.143721
3,Башкортостан,Аскинский,80604000,2009,5.10,0.00,0.0,2600.0,1666.0,0.00,...,0.60,0.0,200.0,1600.0,1944.0,0.00,-3479.0,8.50,428891.0,0.929411
4,Башкортостан,Учалинский,80653000,2009,3.50,0.00,0.0,12700.0,1017.1,32.70,...,10.40,0.0,0.0,0.0,1168.6,25.20,6675.0,0.00,323221.0,0.530328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,Магаданская,Магадан,44701000,2012,109.50,0.00,0.0,2500.0,1239.0,29.80,...,1.14,0.0,0.0,244.0,0.0,0.00,-32989.0,0.00,966.0,3.185414
151,Мордовия,Саранск,89701000,2012,0.00,0.00,100.0,9800.0,0.0,22.10,...,0.00,0.0,0.0,0.0,0.0,0.00,199763.0,0.80,9335.0,0.967685
152,Карелия,Сегежский,86645000,2012,21.80,0.00,100.0,33.0,1470.0,13.93,...,64.60,0.0,0.0,400.0,1431.5,0.00,-2649.0,0.40,802898.0,0.566657
153,Чувашская,Чебоксарский,97644000,2012,11.20,41.61,0.0,5200.0,766.0,17.00,...,20.50,96.2,0.0,1000.0,525.0,20.00,1804.0,11.00,51675.0,0.658054


In [19]:
df_2012 = pd.read_csv('прототип_данных_2010t5+.csv')
df_2011 = pd.read_csv('без_протестов2008_2010t5+.csv')

df_2012 = df_2012[df_2012["mahalanobis_distance"] <= 10].reset_index(drop=True)

merged_df = pd.merge(
    df_2012,
    df_2011,
    left_on=['municipality_protest', 'oktmo_protest'],
    right_on=['municipality', 'oktmo'],
    how='inner'
)

from2012 = merged_df[[
    'municipality_protest', 'oktmo_protest',
    'Жилье (1000 м2)_protest', 'Дороги (%)_protest',
    'Канализация (1 м)_protest', 'Водопровод (1 метр)_protest',
    'municipality_event', 'oktmo_event',
    'Жилье (1000 м2)_event', 'Дороги (%)_event',
    'Канализация (1 м)_event', 'Водопровод (1 метр)_event', 'fprotets_year'
]]



from2012.to_csv('связка_махаланобис_2010t5+.csv', index=False)
from2012

,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,fprotets_year
0,Абакан,95701000,2.79,0.00,400.0,860.0,Саяногорск,95708000,5.60,0.0,0.0,139.0,2009
1,Таштыпский,95625000,10830.00,0.00,0.0,10.0,Орджоникидзевский,95620000,34717.00,0.0,0.0,1600.0,2010
2,Учалинский,80653000,3.50,0.00,0.0,12700.0,Мелеузовский,80641000,10.40,0.0,0.0,0.0,2009
3,Благовещенск,80615101,0.00,0.00,0.0,3500.0,Старотуймазинский,80651455,0.00,0.0,0.0,0.0,2010
4,Аургазинский,80605000,2.10,12.00,0.0,30100.0,Бакалинский,80607000,1.90,0.0,0.0,1000.0,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,Магадан,44701000,109.50,0.00,0.0,2500.0,Эвенск,44607151,1.14,0.0,0.0,244.0,2012
116,Саранск,89701000,0.00,0.00,100.0,9800.0,Большеберезниковское,89610405,0.00,0.0,0.0,0.0,2012
117,Сегежский,86645000,21.80,0.00,100.0,33.0,Кемский,86612000,64.60,0.0,0.0,400.0,2012
118,Чебоксарский,97644000,11.20,41.61,0.0,5200.0,Козловский,97619000,20.50,96.2,0.0,1000.0,2012


In [20]:
df = pd.read_csv('общие_данные_2010t5+.csv')

df = df[['year', 'municipality', 'oktmo', 'Жилье (1000 м2)', 'Дороги (%)', 'Канализация (1 м)', 'Водопровод (1 метр)', 
         'region_name',
        ]]


df = df.drop_duplicates(subset=['year', 'municipality', 'oktmo'])
df = df[df['year'].isin([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])]



events = df
from2012 = pd.read_csv('связка_махаланобис_2010t5+.csv')
final = pd.DataFrame()

for _, link_row in from2012.iterrows():
    
    protest_filter = (
        (events['municipality'] == link_row['municipality_protest']) &
        (events['oktmo'] == link_row['oktmo_protest'])
    )
    protest_data = events[protest_filter].add_suffix('_protest')
    
    event_filter = (
        (events['municipality'] == link_row['municipality_event']) &
        (events['oktmo'] == link_row['oktmo_event'])
    )
    event_data = events[event_filter].add_suffix('_event')
    
    merged = protest_data.merge(
        event_data,
        left_on='year_protest',
        right_on='year_event',
        how='left'
    ).rename(columns={'year_protest': 'year'})
    

    merged = merged.drop('year_event', axis=1)
    merged['fprotets_year'] = link_row.get('fprotets_year', pd.NA)
    merged.loc[~pd.to_numeric(merged['year'], errors='coerce').astype('Int64').isin([2009, 2010, 2011, 2012]),
               'fprotets_year'] = pd.NA
    final = pd.concat([final, merged], ignore_index=True)


final.to_csv('2010t5+.csv', index=False)
final


,year,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,region_name_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,region_name_event,fprotets_year
0,2008,Абакан,95701000,21.10,0.00,381.0,6300.0,Хакасия,Саяногорск,95708000.0,7.53,0.0,0.0,132.0,Хакасия,NaN
1,2009,Абакан,95701000,2.79,0.00,400.0,860.0,Хакасия,Саяногорск,95708000.0,5.60,0.0,0.0,139.0,Хакасия,2009.0
2,2010,Абакан,95701000,2800.00,0.00,0.0,500.0,Хакасия,Саяногорск,95708000.0,6000.00,0.0,0.0,416.0,Хакасия,2009.0
3,2011,Абакан,95701000,2100.00,0.00,0.0,1400.0,Хакасия,Саяногорск,95708000.0,6000.00,0.0,0.0,0.0,Хакасия,2009.0
4,2012,Абакан,95701000,2.10,0.00,180.0,1946.0,Хакасия,Саяногорск,95708000.0,6.00,0.0,0.0,496.9,Хакасия,2009.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1544,2016,Элиста,85701000,0.00,51.16,23.0,4573.0,Калмыкия,Комсомольское,85642412.0,0.00,0.0,0.0,25390.0,Калмыкия,NaN
1545,2017,Элиста,85701000,0.00,60.24,520.0,4953.0,Калмыкия,Комсомольское,85642412.0,0.00,0.0,0.0,25400.0,Калмыкия,NaN
1546,2018,Элиста,85701000,0.00,11.36,74.0,2617.0,Калмыкия,Комсомольское,85642412.0,0.00,0.0,0.0,25400.0,Калмыкия,NaN
1547,2019,Элиста,85701000,0.00,22.84,132.0,4782.0,Калмыкия,Комсомольское,85642412.0,0.00,0.0,0.0,0.0,Калмыкия,NaN
